In [1]:
# Predict the model, get the model from mlflow registery and predict.

import mlflow
import dagshub
import pandas as pd

# Initialize the dagshub repo information.
dagshub.init(repo_owner='pavipd495', repo_name='spam_or_ham_fsec', mlflow=True)


# Model to pull.
model_name = "Bidirectional_lstm"
model_version = 1
model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")


Accessing as pavipd495

Initialized MLflow to track repo "pavipd495/spam_or_ham_fsec"

Repository pavipd495/spam_or_ham_fsec initialized!

2024-12-03 11:05:30.823585: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 11:05:30.939825: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-03 11:05:31.033753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733204131.132799   11371 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733204131.159905   11371 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 11:05:31.395673: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [8]:
import pickle

# Do pre-processing on hte data.
with open("/home/pavithra/projects/spam_or_ham_fsec/core/libs/tranformers/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)
with open("/home/pavithra/projects/spam_or_ham_fsec/core/libs/tranformers/pca.pkl", "rb") as f:
    pca = pickle.load(f)

# predict.
test_data = pd.read_csv("test_data.csv", header=None)
test_data = scaler.transform(test_data)
test_data = pca.transform(test_data)

y_pred = model.predict(test_data)


/home/pavithra/projects/spam_or_ham_fsec/spam_ham_env/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/pavithra/projects/spam_or_ham_fsec/spam_ham_env/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator PCA from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


40/40 ━━━━━━━━━━━━━━━━━━━━ 39s 983ms/step


In [10]:
y_pred_binary = (y_pred >= 0.5).astype(int)

# The prediction will be in 0 and 1. Change it back to original values. 0 to 1 and 1 to -1.
y_pred_binary = pd.DataFrame(y_pred_binary)
y_pred_binary.replace(1, -1, inplace=True)
y_pred_binary.replace(0, 1, inplace=True)

In [11]:
# Save the predictions into a file.
y_pred_binary.to_csv("test_labels.csv", index=False)

In [12]:
y_pred_binary.value_counts()

0 
-1    1039
 1     211
Name: count, dtype: int64